In [1]:
import model.utils

inputs = model.utils.load_all_basic_blocks_data("data/x86_64/basic_blocks")
print(inputs)

[Data(x=[4, 5], edge_index=[2, 1]), Data(x=[2, 5], edge_index=[2, 1]), Data(x=[0], edge_index=[0]), Data(x=[1, 5], edge_index=[0]), Data(x=[1, 5], edge_index=[0]), Data(x=[1, 5], edge_index=[0]), Data(x=[0], edge_index=[0]), Data(x=[1, 5], edge_index=[0]), Data(x=[0], edge_index=[0]), Data(x=[1, 5], edge_index=[0]), Data(x=[0], edge_index=[0]), Data(x=[1, 5], edge_index=[0]), Data(x=[0], edge_index=[0]), Data(x=[1, 5], edge_index=[0]), Data(x=[0], edge_index=[0]), Data(x=[1, 5], edge_index=[0]), Data(x=[0], edge_index=[0]), Data(x=[1, 5], edge_index=[0]), Data(x=[0], edge_index=[0]), Data(x=[1, 5], edge_index=[0]), Data(x=[0], edge_index=[0]), Data(x=[1, 5], edge_index=[0]), Data(x=[0], edge_index=[0]), Data(x=[1, 5], edge_index=[0]), Data(x=[0], edge_index=[0]), Data(x=[1, 5], edge_index=[0]), Data(x=[0], edge_index=[0]), Data(x=[1, 5], edge_index=[0]), Data(x=[0], edge_index=[0]), Data(x=[1, 5], edge_index=[0]), Data(x=[0], edge_index=[0]), Data(x=[1, 5], edge_index=[0]), Data(x=[0],

In [3]:
measured_cycles = []

In [4]:
input_size = 7 # By the number of features extracted with llvm-mc-embed
hidden_size = 128
output_size = 10
learning_rate = 0.001

In [5]:
from model.GraphEncoder import Encoder
from model.Decoder import Decoder
from model.RLAgent import RlAgent

encoder = Encoder(input_size, hidden_size)
decoder = Decoder(hidden_size, output_size)
agent = RlAgent(encoder, decoder)

In [6]:
import torch.nn as nn
import torch.optim as optim

optimizer = optim.Adam(agent.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

In [ ]:
import random
import torch
import os
from model.reward import reward_function

# Check if a GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the models to the device
encoder = encoder.to(device)
decoder = decoder.to(device)
agent = agent.to(device)

checkpoint_dir = "checkpoints/ryzen"

checkpoint_freq = 5

# Load the most recent checkpoint if it exists
latest_checkpoint = None
if os.path.exists(checkpoint_dir):
    checkpoint_files = sorted(os.listdir(checkpoint_dir), reverse=True)
    if checkpoint_files:
        latest_checkpoint = os.path.join(checkpoint_dir, checkpoint_files[0])

if latest_checkpoint:
    start_epoch = model.utils.load_checkpoint(latest_checkpoint, agent, optimizer)
else:
    start_epoch = 0


num_epochs = 100

data = zip(inputs, measured_cycles)

for i in range(start_epoch, num_epochs):
    bb, measured = random.choice(data)
    input_sequence = bb.x.to(device)
    edge_index = bb.edge_index.to(device)

    hidden = encoder.init_hidden().to(device)

    port_pressures = agent(input_sequence, edge_index, hidden)
    predicted_cycles = model.utils.estimate_cycles(port_pressures)

    reward = reward_function(port_pressures, predicted_cycles, measured, bb.x)

    loss = -torch.sum(port_pressures * reward)
    loss.backward()
    optimizer.step()

    if (i + 1) % checkpoint_freq == 0:
        model.utils.save_checkpoint(i + 1, agent, optimizer, checkpoint_dir)
